# Grouping

Chapter 9 in Python for Data Analysis demonstrates a variety of methods to analyze data via data aggregation and grouping operations. Those are the focus of this session.  Our overall goal for this session is to do Exploratory Data Analysis, which is essentially looking at and probing our data to learn about the patterns we can discover in them.  Often this can generate a better understanding of problems in the data, as well as revealing relationships that might be worth exploring in more depth.


For this exercise, we will use 2010 US Census data, for all census tracts in Kentucky.  A good starting point for any kind of Census data is the American Fact Finder: 

https://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml

The Census Bureau produces several products, the most famous of which is the decennial census, which as its name implies, is a true Census.  There are a limited number of data fields for the whole population (or at least those that the Census workers are able to reach, which is pretty good).  The Census has other products that are surveys, such as the American Community Survey (ACS) which asks more detailed questions on an annual basis from a 1% sample of households.  

In this case, we are working with file DP-1: Profile of General Population and Housing Characteristics: 2010, which is also a part of something called Summary File 1 (SF-1).  It is for the 100% sample.  A data dictionary for selected fields is below.  

In [6]:
import pandas as pd

# skip the second row, which contains descriptions
sf1 = pd.read_csv('data/DEC_10_SF1_combined.csv', skiprows=[1])

# make the tract, county and state separate
sf1['tract'] = sf1['GEO.display-label'].apply(lambda x: x.split(sep=',')[0])
sf1['county'] = sf1['GEO.display-label'].apply(lambda x: x.split(sep=',')[1])
sf1['state'] = sf1['GEO.display-label'].apply(lambda x: x.split(sep=',')[2])

# compute some derived fields
sf1['pct_rent'] = sf1['H4-D004'] / sf1['H4-D001'] * 100
sf1['pct_black'] = sf1['P3-D003'] / sf1['P3-D001'] * 100
sf1['pct_asian'] = sf1['P3-D005'] / sf1['P3-D001'] * 100
sf1['pct_white'] = sf1['P3-D002'] / sf1['P3-D001'] * 100
sf1['pct_hisp'] = sf1['P4-D003'] / sf1['P4-D001'] * 100
sf1['pct_vacant'] = sf1['H5-D001'] / sf1['H1-D001'] * 100
sf1[:8]

,GEO.id,GEO.id2,GEO.display-label,P1-D001,P3-D001,P3-D002,P3-D003,P3-D004,P3-D005,P3-D006,...,H5-D008,tract,county,state,pct_rent,pct_black,pct_asian,pct_white,pct_hisp,pct_vacant
0,1400000US21001970100,21001970100,"Census Tract 9701, Adair County, Kentucky",1727,1727,1683,14,1,0,1,...,60,Census Tract 9701,Adair County,Kentucky,17.411402,0.810654,0.000000,97.452229,2.142444,16.794872
1,1400000US21001970200,21001970200,"Census Tract 9702, Adair County, Kentucky",1722,1722,1635,41,5,0,0,...,35,Census Tract 9702,Adair County,Kentucky,19.252874,2.380952,0.000000,94.947735,2.613240,25.161290
2,1400000US21001970300,21001970300,"Census Tract 9703, Adair County, Kentucky",3016,3016,2944,6,11,8,0,...,106,Census Tract 9703,Adair County,Kentucky,20.521173,0.198939,0.265252,97.612732,1.856764,15.193370
3,1400000US21001970401,21001970401,"Census Tract 9704.01, Adair County, Kentucky",4070,4070,3716,237,1,16,1,...,109,Census Tract 9704.01,Adair County,Kentucky,37.215909,5.823096,0.393120,91.302211,1.547912,10.795743
4,1400000US21001970402,21001970402,"Census Tract 9704.02, Adair County, Kentucky",4261,4261,3950,180,16,16,3,...,70,Census Tract 9704.02,Adair County,Kentucky,30.911681,4.224360,0.375499,92.701244,1.900962,11.642542
5,1400000US21001970500,21001970500,"Census Tract 9705, Adair County, Kentucky",2457,2457,2402,16,2,5,0,...,104,Census Tract 9705,Adair County,Kentucky,17.972832,0.651201,0.203500,97.761498,0.773301,17.641997
6,1400000US21001970600,21001970600,"Census Tract 9706, Adair County, Kentucky",1403,1403,1366,9,0,0,0,...,54,Census Tract 9706,Adair County,Kentucky,16.920474,0.641483,0.000000,97.362794,1.282965,13.848397
7,1400000US21003920100,21003920100,"Census Tract 9201, Allen County, Kentucky",3185,3185,3114,16,8,4,0,...,96,Census Tract 9201,Allen County,Kentucky,15.139752,0.502355,0.125589,97.770801,1.098901,27.559055


## Groupby and Aggregation Operations

Groupby is a powerful method in pandas that follows the split-apply-combine approach to data.  As shown in Figure 9-1 in the context of a sum operation, the data is first split into groups that share the same key values.  Then an operation, in this case a sum, is applied to each group.  Then the results are combined.

The built-in aggregation methods available for groupby operations include:
* count
* sum
* mean
* median
* std, var
* min, max
* first, last

You can also apply your own functions as aggregation methods.

![Groupby Operations](groupby.png "Groupby")

Let's apply this approach to computing total population in each county in our dataset.  We can do this in two steps to help explain what is happening.  First we create a groupby object, using county codes to group all the census blocks in sf1 into groups that share the same county code.

In [ ]:
grouped = sf1['P1-D001'].groupby(sf1['county'])
grouped

Now that we have this grouping object that represents the **split** part of the workflow in the figure above, we can **apply** operations and **combine** the results using methods like sum:

In [ ]:
grouped.sum()

We might want to capture the result in a DataFrame if we want to use it in other processing, like merging the results to the original DataFrame.

In [ ]:
county_pop = sf1['P1-D001'].groupby(sf1['county']).sum().to_frame(name='county_population')
county_pop

Here we merge the county total population with sf1 and create a new DataFrame.

In [ ]:
sf2 = pd.merge(sf1,county_pop, left_on='county', right_index=True)
sf2[:5]

## Transforming Data with Groupby

In some cases you may want to apply a function to your data, by group.  An example would be to normalize a column by a mean of each group.  Say we wanted to subtract the mean population density of each county from the population density of each census block. We could write a function to subtract the mean from each value, and then use the transform operation to apply this to each group:

In [ ]:
def demean(arr):
    return arr - arr.mean()

Now we can apply this tranformation to columns in our dataframe.  As examples, let's 'demean' the pct_black and pct_rent columns, subtracting the county-wide mean of these values from the tract-specific values, so that the result is transformed to have a mean of zero within each county.

To check the results, we print the means per county, then the original values for the first 5 rows, then the transformed results.  The transformed results we should be able to calculate by subtracting the appropriate county mean from the tract value.

In [ ]:
normalized = sf1[['pct_black', 'pct_rent']].groupby(sf1['county']).transform(demean)
print(sf1[['pct_black', 'pct_rent']].groupby(sf1['county']).mean())
print(sf1[['county','pct_black', 'pct_rent']][:5])
print(normalized[:5])

We can merge these transformed results on to the original DataFrame, and check the means of the original variables and the tranformed ones.  The transformed ones should be arbitrarily close to zero.

In [ ]:
sf2 = pd.merge(sf1,normalized, left_index=True, right_index=True)

sf2.groupby('county')[['pct_black_x', 'pct_black_y', 'pct_rent_x', 'pct_rent_y']].mean()

Apply is a method we have learned previously, which allows us to apply a function to each row in a DataFrame.  We can also combine apply with groupby to apply functions based on group membership.  For example, the function 'top' sorts an array and selects the top n rows from it.  We provide some defaults for the arguments of how many rows, and the column to use for the selection:

In [ ]:
def top(df, n=5, column='pct_rent'):
    return df.sort_values(by=column, ascending=False).head(n)

Using this on the full dataset and setting the number of rows and the column to get the top values for, in this case using pct_rent to override the default argument, we get the top 10 tracts in the region in terms of percentage rental.

In [ ]:
top(sf1, n=10, column='pct_rent')

Below we apply this with groupby and use the defaults for n and column, and it applies the function within each county and concatenates the results, producing the top 5 blocks on pop_sqmi for each county in the region.

In [ ]:
sf1.groupby('county').apply(top)

Here we pass arguments to the function to set n and the column to select the top value from.

In [ ]:
sf1.groupby('county').apply(top, n=1, column='P1-D001')

## Exploring Correlations in the Data

Pandas provides simple ways of computing correlation coefficients among the columns in your DataFrame.  If you use corr() on a full DF, it will produce a large correlation table.  A bit hard to navigate and you mostly would not be interested in some of these results.

In [ ]:
sf1.corr()

It is easy to compute correlation coeffients for a subset of columns.

In [ ]:
sf1[['pct_rent', 'pct_vacant']].corr()

And this method can be combined with groupby to compute correlation tables by group.

In [ ]:
sf1.groupby('county')[['pct_rent', 'pct_vacant']].corr()

## Your turn to practice:

Count the number of census blocks per county.

Calculate total households per county.

Calculate percent renters by county. (Careful not to calculate the mean percent rental across blocks in a county)

Calculate percent vacant by county.

Calculate mean, min and max vacancy rate (at the block level) by county.

Calculate the 90th percentile of vacancy rate (at the block level) by county.

In [7]:
#Count the number of census blocks per county
census_blocks_per_county_count = sf1["GEO.id"].groupby(sf1['county']).count().to_frame(name='census_blocks_per_county_count')
census_blocks_per_county_count

,census_blocks_per_county_count
county,
Adair County,7
Allen County,6
Anderson County,5
Ballard County,3
Barren County,10
...,...
Wayne County,5
Webster County,4
Whitley County,8


In [8]:
#Calculate total households per county. Households = housing_units?
total_households_per_county = sf1["H1-D001"].groupby(sf1['county']).sum().to_frame(name='total_households_per_county')
total_households_per_county

,total_households_per_county
county,
Adair County,8568
Allen County,9307
Anderson County,9127
Ballard County,3885
Barren County,19188
...,...
Wayne County,10942
Webster County,5936
Whitley County,15166


In [9]:
#Calculate percent renters by county.

renter_df = sf1[['H3-D001', 'H4-D004']].groupby(sf1['county']).sum()
renter_df['percent_renters_by_county'] = renter_df['H4-D004'] / renter_df['H3-D001'] * 100
renter_df
#sf1['pct_rent'] = sf1['H4-D004'] / sf1['H4-D001'] * 100
# percent_renters_by_county = sf1["H1-D001"].groupby(sf1['county']).sum().to_frame(name='percent_renters_by_county')
# percent_renters_by_county

,H3-D001,H4-D004,percent_renters_by_county
county,,,
Adair County,8568,1860,21.708683
Allen County,9307,1898,20.393252
Anderson County,9127,2067,22.647091
Ballard County,3885,727,18.712999
Barren County,19188,5449,28.397957
...,...,...,...
Wayne County,10942,2386,21.805886
Webster County,5936,1245,20.973720
Whitley County,15166,4121,27.172623


In [10]:
#Calculate percent vacant by county.



vacant_df = sf1[['H3-D001', 'H3-D003']].groupby(sf1['county']).sum()
vacant_df['percent_vacant_by_county'] = vacant_df['H3-D003'] / vacant_df['H3-D001'] * 100
vacant_df

#sf1['pct_rent'] = sf1['H4-D004'] / sf1['H4-D001'] * 100
# percent_vacant_by_county =
# percent_vacant_by_county

,H3-D001,H3-D003,percent_vacant_by_county
county,,,
Adair County,8568,1283,14.974323
Allen County,9307,1459,15.676373
Anderson County,9127,758,8.305029
Ballard County,3885,488,12.561133
Barren County,19188,2189,11.408172
...,...,...,...
Wayne County,10942,2296,20.983367
Webster County,5936,664,11.185984
Whitley County,15166,1591,10.490571


In [11]:
#Calculate mean, min and max vacancy rate (at the block level) by county.
# vacancy_rate_stats_mean_min_max_by_county =
# vacancy_rate_stats_mean_min_max by_county

mean_vacant_df = sf1[['pct_vacant']].groupby(sf1['county']).mean()
min_vacant_df = sf1[['pct_vacant']].groupby(sf1['county']).min()
max_vacant_df = sf1[['pct_vacant']].groupby(sf1['county']).max()
 
vacancy_rate_stats_mean_min_max_by_county = pd.merge(min_vacant_df, mean_vacant_df, left_on='county', left_index=True, right_index=True)
vacancy_rate_stats_mean_min_max_by_county = pd.merge(vacancy_rate_stats_mean_min_max_by_county, max_vacant_df, left_on='county', left_index=True, right_index=True)
vacancy_rate_stats_mean_min_max_by_county
vacancy_rate_stats_mean_min_max_by_county.columns = ['min', 'mean', 'max']
vacancy_rate_stats_mean_min_max_by_county

# new_df = sf1[['H4-D001', 'H4-D003', "H4-D004"]].groupby(sf1['county']).sum()
# new_df
# new_df['percent_vacant_by_county'] = vacant_df['H4-D003'] / vacant_df['H4-D001'] * 100
# vacant_df

,min,mean,max
county,,,
Adair County,10.795743,15.868316,25.161290
Allen County,9.254975,15.808521,27.559055
Anderson County,5.492297,8.927276,13.452028
Ballard County,11.608392,12.640382,14.648603
Barren County,7.457767,11.444842,22.975207
...,...,...,...
Wayne County,9.452736,18.476934,41.228851
Webster County,9.475375,11.019754,13.524590
Whitley County,8.378044,10.906092,13.972810


In [13]:
min_vacant_df

,pct_vacant
county,
Adair County,10.795743
Allen County,9.254975
Anderson County,5.492297
Ballard County,11.608392
Barren County,7.457767
...,...
Wayne County,9.452736
Webster County,9.475375
Whitley County,8.378044


In [12]:
#Calculate the 90th percentile of vacancy rate (at the block level) by county.
vacancy_rate_stats_ninety_percentile_by_county = sf1[['pct_vacant']].groupby(sf1['county']).quantile(.9)
vacancy_rate_stats_ninety_percentile_by_county.columns = ['90th_percentile']
vacancy_rate_stats_ninety_percentile_by_county

,90th_percentile
county,
Adair County,20.649714
Allen County,22.324254
Anderson County,12.075984
Ballard County,14.051713
Barren County,15.985510
...,...
Wayne County,31.434558
Webster County,12.664168
Whitley County,12.706375


## Some review:

I've included in the data folder a shapefile with the Census geographies.  Can you use it to: 

1. Calculate the population density of each Census tract and county in Kentucky. 
2. Make a choropleth showing the population density.  

In [ ]:
#import geopndas as gpd
#name=gpd.read_file('data//')

In [ ]:
#merge with precious table
#divide ppuation over area